In [2]:
import torch,pdb
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from tqdm.auto import tqdm
import matplotlib.pyplot as plt


In [3]:
#Visualiztion function

def show(tensor,ch = 1, size=(28,28),num=16):#ch=1 grayscale , num -> number of images you want to show
    data=tensor.detach().cpu().view(-1,ch,size) #detach from computational graphs(remove variables and operations performed earlier\
    #.view -> restructure
    grid = make_grid(data[:num],nrows=4).permute(1,2,0)#making the channel last dimension h,w,ch
    plt.imshow(grid)
    plt.show()

In [5]:
epochs = 500
cur_step = 0
info_step = 300
mean_gen_loss = 0
mean_disc_loss = 0

z_dim = 64
lr = 0.0001
loss_fn = nn.BCEWithLogitsLoss() #BCE with sigmoid

batches = 128
device = "mps" if torch.backends.mps.is_available() else "cpu"


In [8]:
dataloader = DataLoader(MNIST('.',download=True,transform=transforms.ToTensor()),shuffle = True,batch_size = batches)

In [9]:
#Generator

def genBlock(inp,out):
    return nn.Sequential(
        nn.Linear(inp,out),
        nn.BatchNorm1d(out),
        nn.ReLU(inplace=True)
    )



class Generator(nn.Module):
    def __init__(self, z_dim=64,i_dim = 28*28,h_dim = 128) -> None: #i_dim -> output image dimension , hidden layer size -> 128,z_dim -> noise
        super().__init__()
        self.gen = nn.Sequential(
            genBlock(z_dim,h_dim),
            genBlock(h_dim, h_dim*2),
            genBlock(h_dim*2, h_dim*4),
            genBlock(h_dim*4,h_dim*8),
            nn.Linear(h_dim*8,i_dim),
            nn.Sigmoid()#between 0 and one , pixel values
        )

        def forward(self,noise):
            return self.gen(noise)


def gen_noise(number,z_dim):
    return torch.randn(number,z_dim).to(device)



In [10]:
def discBlock(inp,out):
    return nn.Sequential(
        nn.Linear(inp,out),
        nn.LeakyReLU(0.2),
    )

class Discriminator(nn.Module):
    def __init__(self,i_dim = 28*28, h_dim=256) -> None:
        super().__init__()
        self.disc= nn.Sequential(
            discBlock(i_dim,h_dim*4), #Discriminator starts with high h_dim
            discBlock(h_dim*4,h_dim*2),
            discBlock(h_dim*2,h_dim),
            nn.Linear(h_dim,1),
            
        )

    def forward(self,image):
        return self.disc(image)
    
        
    
        